In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!pip install transformers==4.3.3
!pip install stop_words
!pip install symspellpy
!pip install language_detector 
!pip install cached_property
!pip install sentencepiece
!pip install config
!pip install umap
!pip install sentence-transformers==0.4.1
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

     |████████████████████████████████| 1.9 MB 8.1 MB/s 
     |████████████████████████████████| 895 kB 57.0 MB/s 
     |████████████████████████████████| 3.3 MB 66.4 MB/s 
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32912 sha256=709e7662448be0456aec3e145d30798f596eb9bffbb9f74e02736f2f25a6e836
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words
     |████████████████████████████████| 2.6 MB 9.3 MB/s 
  Created wheel for language-detector: filename=language_detector-5.0.2-py3-none-any.whl size=7053 sha256=2d9d7d1ce71e0d2f5dd88bd62fb2c693c24d37958eed6f746c59495bd6fdd902
  Stored in directory: /root/.cache/pip/wheels/12/40/73/a0765d65e793332b79dfe6c34c713e7c0066ea785191b3f50a
Successfully built language-detector
     |████████████████████████████████| 1.2 MB 8.1 MB/s 
  Created wheel for umap: filename=umap-0.1.1-py3-none-any.whl size=3564 sha256=b15bae15ce5cfd39442

True

In [8]:
root_path = "/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2"
import sys, os
import config

config.root_path = os.path.abspath(root_path)
sys.path.insert(0, config.root_path)

In [9]:
import tensorflow as tf

from tensorflow.python import keras
import json
import pandas as pd
import numpy as np
import os

from src.encoders.context_encoder_ldabert_2 import ContextEncoderComplex
from src.dataset.ldabert_2 import LDABERT2Dataset
import transformers
import sentence_transformers

from utils.metrics import windowdiff, pk

Downloading:   0%|          | 0.00/630 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/409 [00:00<?, ?B/s]

In [10]:
tf.__version__, transformers.__version__, sentence_transformers.__version__

('2.6.0', '4.3.3', '0.4.1')

## LDA BERT 2 Dataset

In [61]:
dataset_type = "clinical"
dataset = LDABERT2Dataset(dataset_type=dataset_type,
                           pct_data=0.20,
                          max_seq_length=128,
                           max_segment_length=15,
                           augment_pct=0)

sentences, tokenized_sentences, labels = dataset.process()

# vectors_path = '../data/clinical_vectors/lda_bert_{}_{}.pkl'.format(dataset_type, len(sentences))
vectors_filename = '{}_{}.pkl'.format(dataset.pct_data, dataset.augment_pct)

saved_vectors, saved_labels, saved_tokens, saved_tokenized_sentences = dataset.get_saved_vectors("test", dataset.dataset_type, vectors_filename)

if len(saved_vectors) == 0:
    saved_vectors, saved_labels, saved_tokens, saved_tokenized_sentences = dataset.create_vectors("test", dataset.dataset_type, vectors_filename)

something went wrong [Errno 2] No such file or directory: '/content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2/data/lda_bert_2/generated_vectors/test/clinical/0.2_0.pkl'
root path /content/drive/MyDrive/SCHOOL/PhD/Code/context-encoder-v2
Preprocessing raw texts ...
sentences length 2211
Preprocessing raw texts. Done!
lda sentences length 2211
Getting vector representations for LDA ...
Getting vector representations for LDA. Done!
saving vectors... 2211 2211 2211


In [62]:
left_input, mid_input, right_input = dataset.format_sentences_tri_input_plus(saved_tokenized_sentences)

In [63]:
lda_left_input, lda_mid_input, lda_right_input = dataset.format_sentences_tri_input(saved_vectors)

In [64]:
saved_tokenized_sentences['input_ids'].shape, saved_vectors.shape

(TensorShape([2211, 128]), (2211, 12))

## Model

In [53]:
model = ContextEncoderComplex(final_dropout=0.5,
                            dense_neurons=256,
                                  lstm_size=256,
                             lstm_dropout_percentage=0.5,
                             cnn_filters=8,
                             cnn_kernel_size=3,
                             pool_size=2,
                          gamma=30,
                          max_sentence_length=128,
                           bert_trainable=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [54]:
checkpoint_filepath = '../models/LDABERT2/complex/clinical-15051-1-pct-1-aug_XWfSe/no-finetune/checkpoint.ckpt'

In [55]:
# try:
model.load_weights(checkpoint_filepath)
# except:
#     print("No checkpoint available.")

In [56]:
saved_tokenized_sentences

{'input_ids': <tf.Tensor: shape=(651, 80), dtype=int32, numpy=
array([[  101, 20118, 28228, ...,     0,     0,     0],
       [  101,  2045,  2089, ...,     0,     0,     0],
       [  101,  1996,  3291, ...,     0,     0,     0],
       ...,
       [  101,  1996,  2561, ...,     0,     0,     0],
       [  101,  3227,  1996, ...,     0,     0,     0],
       [  101,  1996,  2561, ...,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(651, 80), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(651, 80), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}

## Inference

In [67]:
slice_size = 250

total_predictions = ''
total_ground_truth = ''

for i in range(0, saved_vectors.shape[0]//slice_size):
    start = i*slice_size
    end = start + slice_size
    print(start, end)

    sliced_tokenized_sentences = {
        'input_ids': saved_tokenized_sentences['input_ids'][start:end],
        'token_type_ids': saved_tokenized_sentences['token_type_ids'][start:end],
        'attention_mask': saved_tokenized_sentences['attention_mask'][start:end]
    }
    sliced_vectors = saved_vectors[start:end]
    
    left_input, mid_input, right_input = dataset.format_sentences_tri_input_plus(sliced_tokenized_sentences)
    lda_left_input, lda_mid_input, lda_right_input = dataset.format_sentences_tri_input(sliced_vectors)
    
    logits = model([
        left_input, 
        mid_input, 
        right_input, 
        lda_left_input, 
        lda_mid_input, 
        lda_right_input
    ])

    logits_flattened = [float(p) for p in logits]

    predictions = [1 if float(logit) >= 0.9 else 0 for logit in logits_flattened]

    k = 5

    string_predictions = "".join([str(i) for i in predictions])
    string_ground_truth = "".join([str(i) for i in saved_labels[start:end]])
    total_predictions += string_predictions
    total_ground_truth += string_ground_truth
    print(string_predictions)
    print(string_ground_truth)
    overall_windowdiff = windowdiff(string_predictions, string_ground_truth, k)
    overall_pk = pk(string_predictions, string_ground_truth, k)

    print("{},{},{},{}".format(overall_windowdiff, overall_pk, k, k))

total_windowdiff = windowdiff(total_predictions, total_ground_truth, k)
total_pk = pk(total_predictions, total_ground_truth, k)
print("Complete Analysis", "="*25)
print("{},{},{},{}".format(total_windowdiff, total_pk, k, k))

0 250
0000000000000000000000000000000000000010010100001000000000001000000001100110100100000000000000100000000000000010000000000000000000000000000000001000000000000010010000000000000000000100000000000000000010000010000000000000000000000000000000000000000000
1001000000000000001000010000100000000000000100000000010000000000000010000100000000001000010000000010000000000000010000000100000000000000100000000000000100000000000000100100100000000000000100000000000010010000000010000010000000000000010010000000000000
0.6260162601626016,0.573170731707317,5,5
250 500
0001000000000100000000010010010000000000000000000000000000000001000000000000000010000010000000000000000000000000000000000000100000000000000000010000000000100000000000000000000000000000000000000000000000000000100000000000000000000100000000000000000000
010000000000000010000100000000000000100000010000000000000010000000000000010000000000000100000000000000100000000000000100000000000000100000000000000100010000000000000010000000000000010000000000

In [ ]:
[float(p) for p in predictions], saved_labels

([0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0],
 [1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,